In [1]:
import sys

sys.path.append("../../")
import os
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

import pandas as pd
import requests
from dotenv import load_dotenv
from tqdm.notebook import tqdm

from config import settings


DOTENV_PATH = settings.BASE_DIR/".env"
load_dotenv(DOTENV_PATH)

ROOT_DIR = Path("/home/alron/movie-recommender")
MOVIES_FILEPATH = settings.BASE_DIR/'data/movies_metadata.csv'
TMDB_LINKS_FILEPATH = settings.BASE_DIR/'data/links.csv'

In [14]:
movies_df = pd.read_csv(MOVIES_FILEPATH)
movies_df.head(3)

/tmp/ipykernel_19754/4097625669.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(MOVIES_FILEPATH)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [15]:
def get_poster_path(tmdb_id, api_key):
    base_url = "https://api.themoviedb.org/3/movie/"
    endpoint = str(tmdb_id)
    api_url = f"{base_url}{endpoint}?api_key={api_key}"

    try:
        response = requests.get(api_url)
        data = response.json()
        poster_path = data.get("poster_path")

        if poster_path:
            return poster_path
        else:
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [16]:
API_KEY = os.getenv("TMDB_API_KEY")

def get_poster_path_parallel(row):
    tmdb_id = row.id
    return tmdb_id, get_poster_path(tmdb_id, API_KEY)

# Assuming movies_df is your DataFrame
error_tmdb_ids = []
poster_paths = []

# Using ThreadPoolExecutor for parallelization
with ThreadPoolExecutor() as executor:
    # Map each row to the get_poster_path_parallel function
    results = list(tqdm(executor.map(get_poster_path_parallel, movies_df.itertuples()), total=len(movies_df)))

# Iterate over results to populate poster_paths and handle errors
for tmdb_id, poster_path in results:
    if poster_path:
        poster_paths.append(poster_path)
    else:
        print(f"Error retrieving poster URL for {tmdb_id}")
        error_tmdb_ids.append(tmdb_id)
        poster_paths.append(None)

# Update movies_df with poster_paths
movies_df["poster_paths"] = poster_paths

# Save the DataFrame to CSV
movies_df.to_csv(settings.BASE_DIR/"data/movies_metadata_fixed_posters.csv", index=False)

print(f"{len(error_tmdb_ids)} errors:\n{error_tmdb_ids}")


  0%|          | 0/45466 [00:00<?, ?it/s]

Error retrieving poster URL for 139405
Error retrieving poster URL for 290157
Error retrieving poster URL for 124639
Error retrieving poster URL for 316098
Error retrieving poster URL for 123505
Error retrieving poster URL for 27768
Error retrieving poster URL for 277270
Error retrieving poster URL for 131232
Error retrieving poster URL for 314352
Error retrieving poster URL for 42005
Error retrieving poster URL for 282919
Error retrieving poster URL for 124619
Error retrieving poster URL for 241058
Error retrieving poster URL for 410921
Error retrieving poster URL for 356054
Error retrieving poster URL for 348138
Error retrieving poster URL for 331367
Error retrieving poster URL for 49799
Error retrieving poster URL for 55146
Error retrieving poster URL for 203829
Error retrieving poster URL for 102878
Error retrieving poster URL for 251481
Error retrieving poster URL for 293820
Error retrieving poster URL for 83593
Error retrieving poster URL for 143750
Error retrieving poster URL fo